<a href="https://colab.research.google.com/github/Rahul626-stack/Capstone_Project/blob/main/Model_2_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway bokeh panel --quiet

import pandas as pd
import pathway as pw
import datetime
import panel as pn
pn.extension()


In [2]:
# Load dataset
df = pd.read_csv("dataset.csv")

# Combine date and time into Timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort by time
df = df.sort_values('Timestamp').reset_index(drop=True)

# Map vehicle weights
veh_weight_map = {"car": 1.0, "bike": 0.8, "truck": 1.5 ,"cycle": 0.5}
df['veh_weight'] = df['VehicleType'].map(veh_weight_map).fillna(1.0)

# Map traffic nearby
traffic_map = {"low": 0.0, "average": 0.5, "high": 1.0}
df['Traffic'] = df['TrafficConditionNearby'].map(traffic_map).fillna(0.0) # Fill NaN with 0.0

# Select columns for streaming
stream_df = df[[
    "Timestamp", "SystemCodeNumber", "Capacity",
    "Occupancy", "IsSpecialDay", "veh_weight","QueueLength","Traffic"
]]

# Save to CSV
stream_df.to_csv("parking_stream.csv", index=False)

In [3]:
df.sample(10)


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,veh_weight,Traffic
15292,8964,Broad Street,690,26.137958,91.740994,677,car,high,8,0,07-12-2016,13:59:00,2016-12-07 13:59:00,1.0,1.0
2064,8019,Broad Street,690,26.137958,91.740994,485,bike,low,3,0,12-10-2016,09:26:00,2016-10-12 09:26:00,0.8,0.0
854,3997,BHMMBMMBX01,687,20.000035,78.000003,643,truck,average,5,0,07-10-2016,11:30:00,2016-10-07 11:30:00,1.5,0.5
16109,10334,Others-CCCPS105a,2009,26.147473,91.728049,1587,car,low,3,0,10-12-2016,15:59:00,2016-12-10 15:59:00,1.0,0.0
4504,4257,BHMMBMMBX01,687,20.000035,78.000003,480,car,low,4,1,23-10-2016,15:26:00,2016-10-23 15:26:00,1.0,0.0
9573,12491,Others-CCCPS135a,3883,26.147499,91.728005,1420,car,average,3,0,12-11-2016,16:34:00,2016-11-12 16:34:00,1.0,0.5
13215,14063,Others-CCCPS202,2937,26.147491,91.727997,758,car,average,8,1,27-11-2016,11:31:00,2016-11-27 11:31:00,1.0,0.5
8916,12444,Others-CCCPS135a,3883,26.147499,91.728005,3197,car,high,9,0,10-11-2016,11:00:00,2016-11-10 11:00:00,1.0,1.0
16093,10333,Others-CCCPS105a,2009,26.147473,91.728049,1586,cycle,low,3,0,10-12-2016,15:25:00,2016-12-10 15:25:00,0.5,0.0
1222,9271,Others-CCCPS105a,2009,26.147473,91.728049,1421,bike,low,3,0,08-10-2016,15:30:00,2016-10-08 15:30:00,0.8,0.0


In [4]:
class ParkingSchema(pw.Schema):
    Timestamp:         str
    SystemCodeNumber:  str
    Capacity:          float
    Occupancy:         float
    IsSpecialDay:      int
    veh_weight:        float
    QueueLength:       float
    Traffic:           float

source = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=500  # Simulated rows/second
)

# Parse datetime
source = source.with_columns(
    t = source.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)


In [5]:
ALPHA     = 1.0
BETA      = 0.5
GAMMA     = 0.6
DELTA     = 0.6
EPSILON   = 0.4
LAMBDA    = 0.2
BASE_PRICE = 10

hourly_price = (
    source
    .windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        window_end = pw.this._pw_window_end,
        occupancy  = pw.reducers.max(pw.this.Occupancy),
        capacity   = pw.reducers.max(pw.this.Capacity),
        is_special = pw.reducers.max(pw.this.IsSpecialDay),
        veh_weight = pw.reducers.max(pw.this.veh_weight),
        queue_len  = pw.reducers.max(pw.this.QueueLength),
        traffic    = pw.reducers.max(pw.this.Traffic),
        system_id  = pw.reducers.max(pw.this.SystemCodeNumber),
    )
    .with_columns(
        demand = (
            ALPHA   * (pw.this.occupancy / pw.this.capacity)
            + BETA  * pw.this.queue_len
            - GAMMA * pw.this.traffic
            + DELTA * pw.this.is_special
            + EPSILON * pw.this.veh_weight
        )
    )
    .with_columns(
        Price = BASE_PRICE * (1 + LAMBDA * pw.this.demand)
    )
    .select(
        SystemCodeNumber = pw.this.system_id,
        Timestamp        = pw.this.window_end,
        veh_weight       = pw.this.veh_weight,
        demand           = pw.this.demand,
        Price            = pw.this.Price
    )
)


In [6]:
pw.io.csv.write(
    table=hourly_price,
    filename="half_hourly_price_output_M2.csv"
)

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [7]:
import panel as pn
import bokeh.plotting as bp
from bokeh.models import ColumnDataSource

pn.extension()

#  Streaming plot function for real-time price updates
def price_plotter(source: ColumnDataSource):
    fig = bp.figure(
        height=400,
        width=800,
        title="Real-Time Dynamic Parking Price",
        x_axis_type="datetime",
        x_axis_label="Timestamp",
        y_axis_label="Price"
    )

    # Line + dot markers
    fig.line("Timestamp", "Price", source=source, line_width=2, color="firebrick")
    fig.scatter("Timestamp", "Price", source=source, size=5, color="firebrick")

    return fig

#  Connect your demand-based hourly_price table to Bokeh plot
viz = hourly_price.plot(price_plotter, sorting_col="Timestamp")

# ⬇ Show the visualization using Panel
pn.Column(
    "#  Live Dynamic Parking Price",
    viz
).servable()



Column
    [0] Markdown(str)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [8]:
pw.run()


Output()

In [9]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot, column
from bokeh.models import ColumnDataSource, HoverTool, Legend
from bokeh.palettes import Category10

# Enable inline Bokeh output
output_notebook()

#  Load the output CSV from Pathway
df = pd.read_csv("half_hourly_price_output_M2.csv")
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

#  Map veh_weight to readable vehicle types
weight_to_type = {
    0.5: "Cycle",
    0.8: "Bike",
    1.0: "Car",
    1.5: "Truck"
}
df["veh_type"] = df["veh_weight"].map(weight_to_type)

#  Setup for colors and plots
palette = Category10[10]
color_map = {}
plots = []
legend_items = {}

#  Create one plot per vehicle type
for veh in ["Cycle", "Bike", "Car", "Truck"]:
    p = figure(
        width=500,
        height=350,
        title=f"Dynamic Pricing: {veh}",
        x_axis_type="datetime",
        x_axis_label="Time",
        y_axis_label="Price",
        tools="pan,wheel_zoom,box_zoom,reset"
    )

    # Add hover tooltip
    p.add_tools(HoverTool(
        tooltips=[("Time", "@x{%F %H:%M}"), ("Price", "@y")],
        formatters={"@x": "datetime"},
        mode="vline"
    ))

    # Subset data for this vehicle type
    sub_df = df[df["veh_type"] == veh]

    # Draw one line per SystemCodeNumber (parking lot)
    for system_id, group in sub_df.groupby("SystemCodeNumber"):
        group = group.sort_values("Timestamp")
        color = color_map.setdefault(system_id, palette[len(color_map) % len(palette)])

        source = ColumnDataSource(data={
            "x": group["Timestamp"],
            "y": group["Price"]
        })

        line = p.line("x", "y", source=source, line_width=2, color=color)

        # Add to legend only once
        if system_id not in legend_items:
            legend_items[system_id] = (f"Lot {system_id}", [line])

    plots.append(p)

#  Create shared legend
legend_fig = figure(
    width=800,
    height=100,
    title="🅿️ Legend: Parking Lots by SystemCodeNumber",
    toolbar_location=None,
    outline_line_color=None,
    x_range=(0, 1),
    y_range=(0, 1)
)
legend_fig.axis.visible = False
legend_fig.grid.visible = False

#  Add all legend entries
legend = Legend(items=list(legend_items.values()), location="center", orientation="horizontal", label_text_font_size="9pt")
legend_fig.add_layout(legend, 'center')

#  Final layout: 2x2 grid + shared legend
grid = gridplot([plots[i:i+2] for i in range(0, len(plots), 2)])
show(column(grid, legend_fig))
